In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.applications import MobileNetV3Small
import time
keras = tf.keras
layers = tf.keras.layers
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0  # normalize to [0,1] range
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\mobilenetv3\3\mobilenetv3.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
#print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(len(train_image_label_ds))



18935


# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
#print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(len(test_image_label_ds))

4729


In [6]:
MobileNetV3 = MobileNetV3Small(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
MobileNetV3.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
x = MobileNetV3(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
MobilenetV3small (Functional (None, 1, 1, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
out_weight (Dense)           (None, 1)                 1025      
Total params: 1,530,993
Trainable params: 1,518,881
Non-trainable params: 12,112
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
np.savetxt('mobilenetv3_loss3.txt',history.history['loss'])
#np.savetxt('DenseNet201_acc.txt',history.history['acc'])
np.savetxt('mobilenetv3_val_loss3.txt',history.history['val_loss'])
#np.savetxt('DenseNet201_val_acc.txt',history.history['val_acc'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sun Apr  2 10:42:41 2023
Epoch 1/300
591/591 [==============================] - 43s 46ms/step - loss: 52.3285 - val_loss: 3143.4246
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\3\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/300
591/591 [==============================] - 28s 47ms/step - loss: 2.8963 - val_loss: 2709.8333
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\3\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 3/300
591/591 [==============================] - 29s 49ms/step - loss: 1.6301 - val_loss: 2380.9607
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\3\mobilenetv3.ckpt\assets


...

C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 218/300
591/591 [==============================] - 53s 90ms/step - loss: 0.1376 - val_loss: 0.1259
Epoch 219/300
591/591 [==============================] - 53s 90ms/step - loss: 0.1399 - val_loss: 0.1371
Epoch 220/300
591/591 [==============================] - 53s 90ms/step - loss: 0.1477 - val_loss: 0.1507
Epoch 221/300
591/591 [==============================] - 53s 90ms/step - loss: 0.1423 - val_loss: 0.1155
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\3\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 222/300
591/591 [==============================] - 56s 94ms/step - loss: 0.1397 - val_loss: 0.1590
Epoch 223/300
591/591 [==============================] - 55s 93ms/step - loss: 0.1315 - val_loss: 0.1437
Epoch 224/300
591/591 [==============================] - 55s 94ms/step - loss: 0.1396 - val_loss: 0.1250
Epoch 225/300
591/591 [==============================] - 55s 93ms/step - loss: 0.1416 - val_loss: 0.1322
Epoch 226/300
591/591 [==============================] - 55s 94ms/step - loss: 0.1383 - val_loss: 0.1368
Epoch 227/300
591/591 [==============================] - 56s 94ms/step - loss: 0.1433 - val_loss: 0.1355
Epoch 228/300
591/591 [==============================] - 56s 94ms/step - loss: 0.1410 - val_loss: 0.1302
Epoch 229/300
591/591 [==============================] - 56s 94ms/step - loss: 0.1383 - val_loss: 0.1266
Epoch 230/300
591/591 [==============================] - 56s 94ms/step - loss: 0.1465 - val_loss: 0.1309
Epoch 231/300
591/591 [==============================] 

Epoch 300/300
591/591 [==============================] - 58s 99ms/step - loss: 0.1306 - val_loss: 0.1417
end time: Sun Apr  2 15:01:43 2023
